# Notebook to test DESI stack algorithm

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table

import speclite
import desispec.io
import sys
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from astropy.convolution import convolve, Box1DKernel, Gaussian1DKernel
output_notebook()

In [ ]:
my_path = Path("/global/cscratch1/sd/bid13/stack-spectra")

In [ ]:
zcat = Table.read(my_path / "zcat.fits")
spectra = desispec.io.read_spectra(my_path / "spectra_coadd_cam.fits")

In [ ]:
def random_plot_spec(flux, wave, num_plot=1, **args):
    fig = plt.figure(figsize=(20,8))
    if flux.ndim==1:
        plt.plot(wave,flux, **args)
    else:
        num_spec=flux.shape[0]
        if wave.ndim==1:
            wave = np.repeat(wave[np.newaxis,:], num_spec, axis=0)
        idx = np.random.choice(num_spec, size=num_plot, replace=False)
        for count, i in enumerate(idx):
            plt.plot(wave[i],flux[i], lw=0.5)
    plt.ylim(5,-5)
#     return fig

In [ ]:
flux = spectra.flux["brz"]
wave = spectra.wave["brz"]
ivar = spectra.ivar["brz"]
z = zcat["Z"]

In [ ]:
index =1
p = figure(title="Spectra", x_axis_label='Wavelength', y_axis_label='Flux', plot_width=1000, plot_height=300, y_range=(-1, 1))
p.line(wave, flux[index], legend_label="Temp.", line_width=2)
p.line(wave, convolve(flux[index],Gaussian1DKernel(9)), legend_label="Temp.", line_width=2, color="orange")

show(p)

In [ ]:
random_plot_spec(flux, wave, 10)

In [ ]:
from importlib import reload
import sys
sys.path.append("../")

import stack_spectra
reload(stack_spectra)
import spectral_resampling 
reload(spectral_resampling)

In [ ]:
# wavegrid = np.arange(3000, 5000, 0.8)
flux_new, ivar_new, wave_new = stack_spectra._common_grid(flux,wave,ivar,z)

In [ ]:
index =3
p = figure(title="Spectra", x_axis_label='Wavelength', y_axis_label='Flux', plot_width=1000, plot_height=300, y_range=(-1, 1), tooltips=[('Flux', '$x'), ('Wavelength', '$y')])
p.line(wave_new, flux_new[index], legend_label="Temp.", line_width=2)
p.line(wave_new, convolve(flux_new[index],Gaussian1DKernel(2)), legend_label="Temp.", line_width=2, color="orange")

show(p)

In [ ]:
x = np.nanmean(flux_new/np.nanmedian(flux_new, axis=1, keepdims=True),axis=0)
from astropy.convolution import convolve, Box1DKernel
smoothed_signal = convolve(x, Box1DKernel(5))
random_plot_spec(x, wave_new)
plt.ylim(-5,5)
plt.xlim(3900,4000)

In [ ]:
random_plot_spec(flux_new, wave_new, 10)
plt.xlim(3000, 4000)